In [2]:
import bs4
import lxml
import pandas as pd
import urllib
from urllib import request
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil
import re
import numpy as np

In [10]:
#Récupération des liens des pages
url = "http://www.caveaudelahuchette.fr/"
site = request.urlopen(url).read()
page = bs4.BeautifulSoup(site, 'lxml')
div = page.find_all('ul')
menu = div[1]                                                                    #On récupère le code html du menu principal
items = menu.find_all('li',class_= re.compile("concert", flags=re.IGNORECASE))   #On utilise regex pour récupérer tous les lignes qui contiennet le terme "concert"
liens ={}
for item in items:
    liens[str(item.get('class')[0].split('_')[1])] = url + item.a.get('href')
                                                                                  #On a récupéré les liens de la programmation pour chaques mois
print(liens)




{'novembre': 'http://www.caveaudelahuchette.fr//1/concerts_novembre_2023_1482427.html', 'decembre': 'http://www.caveaudelahuchette.fr//1/concerts_decembre_2023_1482445.html', 'janvier': 'http://www.caveaudelahuchette.fr//1/concerts_janvier_2024_1482553.html'}


In [4]:
#Fonction pour changer les dates
def get_days(string):
    figures = re.findall(r'\d+', string)
    if 'au' in string:
        days = np.arange(int(figures[0]),int(figures[1])+1)
    elif 'et' in string:
        days = np.array([int(figures[0]),int(figures[1])])
    else:
        days = np.array([int(figures[0])])
    return days
    

get_days('Mardi 28 novembre au 43')

array([28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43])

In [22]:
#Pour une page
url = "http://www.caveaudelahuchette.fr//1/concerts_novembre_2023_1482427.html"
site = request.urlopen(url).read()
page = bs4.BeautifulSoup(site, 'lxml')
balise_p = [p for p in page.find_all('p') if p.text.strip()]

horaires =(balise_p[1].text.split()[7][:5], balise_p[1].text.split()[24][:5])     #Récupération des heures de concert sous le format (heure en semaine, heure le week-end)

programmation = balise_p[2].get_text(separator = '%', strip = True).split('%')


information = ['nom', 'date','heure','prix','etablissement','adresse','arrondissement']
D ={key:[] for key in information}
for concert in programmation:
    carac = concert.split(' : ')
    days = get_days(carac[0])
    for day in days:
        D['date'].append( str(day) + '/11')
        D['nom'].append(carac[1])

D['prix'] = ['k']*len(D['nom'])
D['etablissement'] = ['Caveau de la Huchette']*len(D['nom'])
D['heure'] =['21h30']*len(D['nom'])
D['adresse'] = ['5 rue de la Huchette']*len(D['nom'])
D['arrondissement'] = ['5']*len(D['nom'])
df = pd.DataFrame(D)
print(df)


                                                  nom   date  heure prix  \
0   Dany Doriz & Michel Pastre "Fathers & Sons" in...   1/11  21h30    k   
1     The Suits Playground Tribute to Louis Armstrong   2/11  21h30    k   
2     The Suits Playground Tribute to Louis Armstrong   3/11  21h30    k   
3                  Matthieu Boré Blues in New Orleans   4/11  21h30    k   
4                                           Megaswing   5/11  21h30    k   
5                                        Blues Monday   6/11  21h30    k   
6                 Nirek Mokar & His Boogie Messengers   7/11  21h30    k   
7                             Five O'Clock Jazz Group   8/11  21h30    k   
8                            FERMETURE EXCEPTIONNELLE   9/11  21h30    k   
9   François Laudet Tribute to Gene Krupa & Buddy ...  10/11  21h30    k   
10  François Laudet Tribute to Gene Krupa & Buddy ...  11/11  21h30    k   
11  François Laudet Tribute to Gene Krupa & Buddy ...  12/11  21h30    k   
12          

In [21]:
#Pour une page
url = "http://www.caveaudelahuchette.fr//1/concerts_decembre_2023_1482445.html"
site = request.urlopen(url).read()
page = bs4.BeautifulSoup(site, 'lxml')
balise_p = [p for p in page.find_all('p') if p.text.strip()]

horaires =(balise_p[1].text.split()[7][:5], balise_p[1].text.split()[24][:5])     #Récupération des heures de concert sous le format (heure en semaine, heure le week-end)

programmation = balise_p[2].get_text(separator = '%', strip = True).split('%')


information = ['nom', 'date','heure','prix','etablissement','adresse','arrondissement']
D ={key:[] for key in information}
for concert in programmation:
    carac = concert.split(' : ')
    days = get_days(carac[0])
    for day in days:
        D['date'].append( str(day) + '/12')
        D['nom'].append(carac[1])

D['prix'] = ['k']*len(D['nom'])
D['etablissement'] = ['Caveau de la Huchette']*len(D['nom'])
D['heure'] =['21h30']*len(D['nom'])
D['adresse'] = ['5 rue de la Huchette']*len(D['nom'])
D['arrondissement'] = ['5']*len(D['nom'])
df = pd.DataFrame(D)
print(df)


[<p><span style="text-decoration: underline;">Heure des concerts / Concerts time</span></p>, <p>Semaine / Week : 1er set : 21h30-22h30 / 2ème set : 23h00-00h00 / 3ème set : 00h30 - 01h30<br/>Week-end : 1er set : 21h30-22h30 / 2ème set : 23h00-00h15 / 3ème set : 00h45 - 01h45</p>, <p>Vendredi 1 et samedi 2 décembre : Charlye Dream<br/>Dimanche 3 décembre : Megaswing <br/>Lundi 4 décembre : Blues Monday<br/>Mardi 5 décembre : Five O'Clock Jazz Group<br/>Du mercredi 6 au Samedi 9 décembre : You Rascal Band<br/>Dimanche 10 et lundi 11 décembre : Nirek Mokar &amp; His Boogie Messengers invite Sax Gordon<br/>Mardi 12 décembre : Blue Waltz<br/>Mercredi 13 décembre : Nirek Mokar &amp; His Boogie Messengers invite Sax Gordon<br/>Du jeudi 14 au samedi 16 décembre : Jack’&amp;’Jills Swing Band<br/>Dimanche 17 décembre : Megaswing <br/>Lundi 18 et mardi 19 décembre : Louis Prima for Ever Special Noël<br/>Mercredi 20 décembre : Dany Doriz &amp; Michel Pastre "Fathers &amp; Sons"<br/>Du jeudi 21 au 

In [110]:
#L'international
url_2 = "https://www.linternational.fr/agenda"
request_text = request.urlopen(url_2).read()
page = bs4.BeautifulSoup(request_text, "lxml")
exhibitions = page.find_all('div', class_ = "Zz5cHg")
information = ['nom', 'date','heure','prix','etablissement','adresse','arrondissement']
D = {key:[] for key in information}
id = 0
for exhibition in exhibitions:
    info = exhibition.img.get('alt').split(' - ')
    D['nom'].append(id)
    D['date'].append(info[0])
    D['heure'].append(info[1])
    D['prix'].append(info[2])
    id = id + 1
n = len(D['date'])

p = []                                 
for prix in D['prix']:
    p.append(prix[:-1])
D['prix'] = p

D['etablissement'] = ["L'international"]*n
D['adresse'] = ["5 rue Moret"]*n
D['arrondissement'] = [11]*n

data = pd.DataFrame(D)
data.head()
#international_csv = data.to_csv('C:\\Users\\LENOVO\\Desktop\\data_international.csv', index =False, encoding = 'utf-8')



,nom,date,heure,prix,etablissement,adresse,arrondissement
0,0,Vendredi 24 novembre,20h,8,L'international,5 rue Moret,11
1,1,Vendredi 24 novembre,00h,7,L'international,5 rue Moret,11
2,2,Samedi 25 novembre,20h,8,L'international,5 rue Moret,11
3,3,Samedi 25 novembre,00h,7,L'international,5 rue Moret,11
4,4,Dimanche 26 novembre,18h,Gratui,L'international,5 rue Moret,11


In [1]:
import folium
from folium.plugins import MarkerCluster

# Créer une carte Folium
m = folium.Map(location=[48.8566, 2.3522], zoom_start=12)  # Coordonnées de Paris

# Créer un objet MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

# Ajouter des marqueurs au MarkerCluster
folium.Marker(location=[48.8566, 2.3522], popup="Description 1").add_to(marker_cluster)
folium.Marker(location=[48.8566, 2.3522], popup="Description 2").add_to(marker_cluster)
folium.Marker(location=[48.8566, 2.3522], popup="Description 3").add_to(marker_cluster)

# Afficher la carte
m.save("ma_carte.html")


In [5]:
# On attaque le vieux belleville
url = 'https://www.le-vieux-belleville.com/programme/'

site = request.urlopen(url)
code_page = bs4.BeautifulSoup(site, 'lxml')
print(type(code_page))

information = ['nom', 'date','heure','prix','etablissement','adresse','arrondissement']
D ={key:[] for key in information}

title_list = code_page.find_all('span', class_ = 'dp_pec_event_title_sp')
date_list = code_page.find_all('span', class_ = 'dp_pec_date_event_time')

for date in date_list:
    if len(date.text.split()) == 4:                                        #On exclut les conditions autres que les concerts"
        D['date'].append('' + date.text.split()[0] + date.text.split()[1] + date.text.split()[2])
        D['heure'].append(date.text.split()[3])
        
for title in title_list:
    if title.text != "Fermeture exceptionnelle":
        D['nom'].append(title.text)

n = len(D['nom'])
D['prix'] = [0]*n
D['etablissement'] = ['Le vieux belleville']*n
D['adresse'] = ['12 rue des Envierges']*n
D['arrondissement'] = [20]*n

df = pd.DataFrame(D)


<class 'bs4.BeautifulSoup'>


In [6]:
url = 'https://1ou2cocktails.com/cocktails/'

site = request.urlopen(url)
print(code_page)

HTTPError: HTTP Error 403: Forbidden